## **class demo**


### part one: one campsite, one day, send email

In [5]:
import time
import smtplib
from email.mime.text import MIMEText
import pandas as pd
from lxml import html
import requests
import re
import time
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# inputs
MONTH = "Mar"
START_DAY = 22
END_DAY = 23
YEAR = 2025
CAMPSITE = "Parsons Landing"

# email info
EMAIL_ADDRESS = os.getenv("EMAIL_ADDRESS")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
TO_EMAIL = os.getenv("TO_EMAIL")
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587


def send_email(availability):
    subject = f"{CAMPSITE} Availability Alert"
    body = f"There are {availability} spots available at {CAMPSITE} on {MONTH} {START_DAY}! Book now."
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_ADDRESS
    msg["To"] = TO_EMAIL
    
    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.sendmail(EMAIL_ADDRESS, TO_EMAIL, msg.as_string())
        print("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")

url = "https://app.fireflyreservations.com/reserve/property/CatalinaIslandCompany"

#set up selenium scraper
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#since there is no more chromedriver.exe, Selenium should auto-detect the integrated driver
driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(1)

#arrival date
arrival_input = driver.find_element(By.CSS_SELECTOR, "input[data-selecteddateid='arrival-date-value']")
driver.execute_script("arguments[0].removeAttribute('readonly')", arrival_input)
arrival_input.click()

#check the displayed month and navigate if needed
while True:
    current_month = driver.find_element(By.CSS_SELECTOR, "th.month").text.strip()
    if f"{MONTH} {YEAR}" in current_month:
        break  # Stop if the correct month is displayed
    next_button = driver.find_element(By.CSS_SELECTOR, "th.next.available")
    next_button.click()

#find and click the correct arrival date
arrival_date_element = driver.find_element(By.XPATH, f"//td[@class='weekend available' and text()='{START_DAY}']")
arrival_date_element.click()
time.sleep(1)  # Ensure the change is registered

#departure date
departure_input = driver.find_element(By.CSS_SELECTOR, "input[data-selecteddateid='departure-date-value']")
driver.execute_script("arguments[0].removeAttribute('readonly')", departure_input)
departure_input.click()

departure_date_element = driver.find_element(By.XPATH, f"//td[contains(@class, 'active start-date active end-date available') and text()='{END_DAY}']")
departure_date_element.click()
time.sleep(1)

#guests
adults_input = driver.find_element(By.NAME, "Adults")
adults_input.click()
adults_input.clear()
adults_input.send_keys(Keys.BACKSPACE)
adults_input.send_keys("2")

#close input
body = driver.find_element(By.TAG_NAME, "body")
body.click()

#wait until the search button is clickable
wait = WebDriverWait(driver, 10)
search_button = wait.until(EC.element_to_be_clickable((By.ID, "search-class-availability-btn")))

#scroll
driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
time.sleep(1)

search_button.click()
time.sleep(1)

#check availability
try:
    campsite_element = driver.find_element(By.XPATH, f"//div[@class='class-name' and text()='{CAMPSITE.upper()}']/ancestor::div[@data-classguid]//div[@class='available-unit-count']")
    availability = campsite_element.text.strip()
    print(f"{CAMPSITE} availability: {availability} spots available.")
    
    #send an email if spots are available
    if availability.isdigit() and int(availability) > 0:
        send_email(availability)
except Exception as e:
    print(f"Could not find availability information for {CAMPSITE}: {e}")

driver.quit()

Parsons Landing availability: 0 spots available.


### part two, simplified: no email send, store campsite names and availabilities to a list

In [6]:
import time
import pandas as pd
from lxml import html
import requests
import re
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# inputs
MONTH = "Mar"
START_DAY = 22
END_DAY = 23
YEAR = 2025
CAMPSITE = "Parsons Landing"

url = "https://app.fireflyreservations.com/reserve/property/CatalinaIslandCompany"

#set up selenium scraper
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#since there is no more chromedriver.exe, Selenium should auto-detect the integrated driver
driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(1)

#arrival date
arrival_input = driver.find_element(By.CSS_SELECTOR, "input[data-selecteddateid='arrival-date-value']")
driver.execute_script("arguments[0].removeAttribute('readonly')", arrival_input)
arrival_input.click()

#check the displayed month and navigate if needed
while True:
    current_month = driver.find_element(By.CSS_SELECTOR, "th.month").text.strip()
    if f"{MONTH} {YEAR}" in current_month:
        break  # Stop if the correct month is displayed
    next_button = driver.find_element(By.CSS_SELECTOR, "th.next.available")
    next_button.click()

#find and click the correct arrival date
arrival_date_element = driver.find_element(By.XPATH, f"//td[@class='weekend available' and text()='{START_DAY}']")
arrival_date_element.click()
time.sleep(1)  # Ensure the change is registered

#departure date
departure_input = driver.find_element(By.CSS_SELECTOR, "input[data-selecteddateid='departure-date-value']")
driver.execute_script("arguments[0].removeAttribute('readonly')", departure_input)
departure_input.click()

departure_date_element = driver.find_element(By.XPATH, f"//td[contains(@class, 'active start-date active end-date available') and text()='{END_DAY}']")
departure_date_element.click()
time.sleep(1)

#guests
adults_input = driver.find_element(By.NAME, "Adults")
adults_input.click()
adults_input.clear()
adults_input.send_keys(Keys.BACKSPACE)
adults_input.send_keys("2")

#close input
body = driver.find_element(By.TAG_NAME, "body")
body.click()

#wait until the search button is clickable
wait = WebDriverWait(driver, 10)
search_button = wait.until(EC.element_to_be_clickable((By.ID, "search-class-availability-btn")))

#scroll
driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
time.sleep(1)

search_button.click()
time.sleep(1)

# Check availability for all campsites
try:
    campsite_elements = driver.find_elements(By.XPATH, "//div[@class='class-details']")
    
    site_names = []
    site_availabilities = []

    for campsite in campsite_elements:
        # Extract campsite name
        name_element = campsite.find_element(By.CLASS_NAME, "class-name")
        campsite_name = name_element.text.strip()
        
        # Find the availability div within the same parent container
        parent_container = campsite.find_element(By.XPATH, "./ancestor::div[@data-classguid]")
        availability_element = parent_container.find_element(By.CLASS_NAME, "available-unit-count")
        availability = availability_element.text.strip()

        site_names += [campsite_name]
        site_availabilities += [availability]

        print(f"{campsite_name} availability: {availability} spots available.")

except Exception as e:
    print(f"Could not retrieve availability information: {e}")

driver.quit()

BLACK JACK availability: 0 spots available.
CATALINA BOAT-IN availability: 1+ spots available.
HERMIT GULCH availability: 1+ spots available.
LITTLE HARBOR availability: 1+ spots available.
PARSONS LANDING availability: 0 spots available.
TWO HARBORS availability: 1+ spots available.


### part three: in class activity

Iterate over the stored values and print all campsites with spots available. Clean availabiltiy values.

In [7]:
site_availabilities

['0', '1+', '1+', '1+', '0', '1+']

In [8]:
site_names

['BLACK JACK',
 'CATALINA BOAT-IN',
 'HERMIT GULCH',
 'LITTLE HARBOR',
 'PARSONS LANDING',
 'TWO HARBORS']

In [ ]:
# print camp sites that have availability

if len(site_names) == len(site_availabilities):

    for site_index in range(len(site_names)):

        if site_availabilities[site_index] != "0":
            print(site_names[site_index])

else:
    print("there's a problem -- the lists aren't the same length")

CATALINA BOAT-IN
HERMIT GULCH
LITTLE HARBOR
TWO HARBORS


In [16]:
site_availabilities

['0', '1+', '1+', '1+', '0', '1+']

In [ ]:
# clean the availabilities (aka remove the plus signs and convert them to integers)

site_availabilities_clean = []

for site in site_availabilities:
    site_availabilities_clean += [int(site.replace("+",""))]

site_availabilities_clean

[0, 1, 1, 1, 0, 1]

In [ ]:
## BAD PRACTICE to change values in a list while you're iterating through it

for site_index in range(len(site_availabilities)):
    site_availabilities[site_index] = site_availabilities[site_index].replace("+","")

site_availabilities

['0', '1', '1', '1', '0', '1']